In [3]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings

from utils.conn_data import load_pickle

warnings.simplefilter(action='ignore', category=FutureWarning)
plt.style.use('ggplot')

In [4]:
outputs_path = os.path.join(os.getcwd(), 'data', 'outputs')

## Simulation 1c

### Dataset

For each covariance in a list of different covariances between graph pairs, we simulate 30 times (n_simulations) a list of 50 (n_graphs) pairs of graphs from the erdos-renyi family of graphs. For each pair of graph, we sample a random variable p from a multivariate gaussian distribution with fixed mean and covariance.

### Model

1) $\textbf{Spectrum}$: For each pair of graphs, compute the spectral radii for each adjacency matrix individually, and compute the Spearman's rank correlation between the spectral radiis for each graph.

2) $\textbf{SDNE}$: For each epoch, for each pair of graphs, compute the embeddings using the SDNE (autoencoder), and compute the Spearman's rank correlation between the vectorial version of the embeddings of each graph.


In [ ]:
ovrd_models = ['spectrum', 'deltacon', 'frobenius'] + [f'spectrumEx_k={k}' for k in range(1, 6)] + [f'procustes_k={k}' for k in range(1, 6)]
dataset_name = "simulation1c"
simulation_names = os.listdir(os.path.join(outputs_path, dataset_name))

all_metrics = []
for simulation_name in simulation_names:


    if ".DS_Store" == simulation_name:
        continue

    for model in os.listdir(os.path.join(outputs_path, dataset_name, simulation_name)):

        if ".DS_Store" == model:
            continue

        # print(model)

        if model in ovrd_models:
            data = load_pickle(os.path.join(outputs_path, dataset_name, simulation_name, model, 'results.pkl'))
            test_data = data["train_test_results"]
        else:
            data = load_pickle(os.path.join(outputs_path, dataset_name, simulation_name, model, 'predictions.pkl'))
            args = load_pickle(os.path.join(outputs_path, dataset_name, simulation_name, model, 'args.pkl'))
            test_data = data["test_predictions"]

        if len(test_data.shape) > 2:
            test_data_df = []
            for i in range(test_data.shape[0]):
                simulation_test_data = test_data[i, :, :]
                simulation_test_data_df = pd.DataFrame(simulation_test_data.cpu().numpy(), columns=["pred", "true"])
                simulation_test_data_df.loc[:, "simulation"] = i

                test_data_df.append(simulation_test_data_df)
            test_data_df = pd.concat(test_data_df, axis=0)
            test_data_df.loc[:, "true"] = [round(x, 2) for x in test_data_df["true"]]
            test_data_df.loc[:, "model"] = model
        else:
            test_data_df = pd.DataFrame(test_data.numpy(), columns=["pred", "true"])
            test_data_df.loc[:, "true"] = [round(x, 2) for x in test_data_df["true"]]
            test_data_df.loc[:, "model"] = model

        # compute aggregated mse and mae
        mse = mean_squared_error(test_data_df["true"], test_data_df["pred"])
        mae = mean_absolute_error(test_data_df["true"], test_data_df["pred"])

        if model not in ovrd_models:
            try:
                alpha = args['args'].alpha
            except AttributeError:
                alpha = 1

            try:
                theta = args['args'].theta
            except AttributeError:
                theta = 1

            try:
                nu = args['args'].nu
            except AttributeError:
                nu = 1

            try:
                gamma = args['args'].gamma
            except AttributeError:
                gamma = 1

            try:
                early_stopping = args['args'].early_stopping
            except AttributeError:
                early_stopping = False

            try:
                gradient_clipping = args['args'].gradient_clipping
            except AttributeError:
                gradient_clipping = False

            try:
                stadardize_losses = args['args'].stadardize_losses
            except AttributeError:
                stadardize_losses = False

            try:
                eigen_loss_type = args['args'].eigen_loss_type
            except AttributeError:
                eigen_loss_type = None

            model_name = args['args'].model_name
            if model_name in ["sdne8oldesgc", "sdne8oldes", "sdne8old", "sdne8oldesgc-gamma", "sdne8oldesgcs"]:
                model_name = "sdne8old"

            tmp_metrics = pd.DataFrame({
                "dataset": args['args'].dataset_name,
                "graph": args['args'].graph_name,
                "model": model_name,
                "eigen_loss": eigen_loss_type,
                # "model_check": args['args'].model_name,
                "epochs": args['args'].epochs,
                "n_hidden": args['args'].n_hidden,
                "n_layers_enc": args['args'].n_layers_enc,
                "n_layers_dec": args['args'].n_layers_dec,
                "alpha": alpha,
                "theta": theta,
                "nu": nu,
                "gamma": gamma,
                "es": 1 if early_stopping else 0,
                "gc": 1 if gradient_clipping else 0,
                "sl": 1 if stadardize_losses else 0,
                "mse": mse,
                "mae": mae,
            }, index=[0])
        else:
            tmp_metrics = pd.DataFrame({
                "dataset": args['args'].dataset_name,
                "graph": args['args'].graph_name,
                "model": model,
                # "model_check": args['args'].model_name,
                "mse": mse,
                "mae": mae,
            }, index=[0])
        all_metrics.append(tmp_metrics)
all_metrics_df = pd.concat(all_metrics, axis=0)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/danieloliveira/Documents/Daniel/codes/graph-corr-embedd/src/data/outputs/simulation1c/erdos_renyi/procustes_k=1/args.pkl'

In [11]:
data.keys()

dict_keys(['train_test_results', 'n_simulations', 'covs'])

In [8]:
args

NameError: name 'args' is not defined

In [2]:
all_metrics_df.loc[all_metrics_df['graph'] == 'erdos_renyi'].drop(["dataset"], axis=1).sort_values(["graph", "mse"])

NameError: name 'all_metrics_df' is not defined

In [7]:
all_metrics_df.loc[all_metrics_df['graph'] == 'watts_strogatz'].drop(["dataset"], axis=1).sort_values(["graph", "mse"])

,graph,model,eigen_loss,epochs,n_hidden,n_layers_enc,n_layers_dec,alpha,theta,nu,gamma,es,gc,sl,mse,mae
0,watts_strogatz,spectrumEx_k=1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.105549,0.254339
0,watts_strogatz,spectrum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.114484,0.269591
0,watts_strogatz,spectrumEx_k=2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.200373,0.334864
0,watts_strogatz,spectrumEx_k=3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.332106,0.442576
0,watts_strogatz,sdne8old,mse,100.0,50.0,1.0,1.0,0.01,1.0,0.00001,100.0,0.0,0.0,0.0,0.343325,0.506873
0,watts_strogatz,sdne8old,norms,100.0,50.0,1.0,1.0,0.01,1.0,0.00001,100.0,0.0,0.0,0.0,0.348422,0.511322
0,watts_strogatz,sdne8old,norms,100.0,50.0,1.0,1.0,0.01,1.0,0.00001,100.0,1.0,1.0,0.0,0.353250,0.515265
0,watts_strogatz,sdne8old,mse,100.0,50.0,1.0,1.0,0.01,1.0,0.00001,100.0,1.0,1.0,0.0,0.355503,0.516960
0,watts_strogatz,sdne8old,mse,100.0,50.0,1.0,1.0,0.01,1.0,0.00001,100.0,1.0,0.0,0.0,0.355875,0.516480
0,watts_strogatz,sdne8old,norms,100.0,50.0,1.0,1.0,0.00,0.0,0.00000,100.0,1.0,0.0,0.0,0.366511,0.523837
